In [1]:
import os
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.backend import mean
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import plot_model
from PIL import Image


from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

Using TensorFlow backend.


In [2]:
#model = load_model('model/yolov3.h5')

In [3]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

In [4]:
def transpose(X):  #用来转置 暂时废弃
    m, n = len(X), len(X[0])
    return [[X[i][j] for i in range(m)] for j in range(n)]
def loadDatadet(infile):
    f=open(infile,'r')
    sourceInLine=f.readlines()
    dataset=[]
    lists=[]
    for line in sourceInLine:
        temp1=line.strip('\n')
        temp2=temp1.split('\t')
        dataset.append(temp2)
    for i in dataset:
        lists.append(i[0].split(" "))
    return lists
def pocessingData(data):
    info=[]
    img_file=[]
    img_file=[data[i][0] for i in range(len(data))]
    for i in range(len(data)):
        box=[]
        #box.append((int(data[i][2])+int(data[i][3]))/832.)
        #box.append((int(data[i][4])+int(data[i][5]))/832.)
        #box.append((int(data[i][3])-int(data[i][2]))/416.)
        #box.append((int(data[i][5])-int(data[i][4]))/416.)
        box.append(int(data[i][2]))
        box.append(int(data[i][3]))
        box.append(int(data[i][4]))
        box.append(int(data[i][5]))
        box.append(int(data[i][1]))
        info.append((img_file[i],box))
    return info
lists='./image_lists/lists.txt'
lists=loadDatadet(lists)
data=pocessingData(lists)
data[0]

('./img/2.jpg', [0, 17, 0, 17, 1])

In [5]:
def data_generator(data, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(data)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            image, box = data[i]# 获取图片和盒子
            image_data.append(np.array(Image.open(image)))# 添加图片
            box_data.append([box])# 添加盒子
            i = (i + 1) % n
        image_data =np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)# 真值
        yield [image_data] + y_true, np.zeros(batch_size)


def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    """
    用于条件检查
    """
    n = len(annotation_lines)# 标注图片的行数
    if n == 0 or batch_size <= 0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)


In [6]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [7]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
                 weights_path='model/yolov3.h5'):
    K.clear_session()  # 清除session
    image_input = Input(shape=(None, None, 3))  # 图片输入格式
    h, w = input_shape  # 尺寸
    num_anchors = len(anchors)  # anchor数量

    # YOLO的三种尺度，每个尺度的anchor数，类别数+边框4个+置信度1
    y_true = [Input(shape=(h // {0: 32, 1: 16, 2: 8}[l], w // {0: 32, 1: 16, 2: 8}[l],
                           num_anchors // 3, num_classes + 5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors // 3, num_classes)  # model
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:  # 加载预训练模型
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)  # 加载参数，跳过错误
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers) - 3)[freeze_body - 1]
            for i in range(num):
                model_body.layers[i].trainable = False  # 将其他层的训练关闭
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss,
                        output_shape=(1,), name='yolo_loss',
                        arguments={'anchors': anchors,
                                   'num_classes': num_classes,
                                   'ignore_thresh': 0.5})(model_body.output + y_true)  # 后面是输入，前面是输出
    model = Model([model_body.input] + y_true, model_loss)  # 模型，inputs和outputs

    return model

In [8]:
lists='./image_lists/lists.txt'
lists=loadDatadet(lists)
data=pocessingData(lists)

log_dir = 'logs/002/'  # 日志文件夹

pretrained_path = 'model/yolov3.h5'  # 预训练模型
anchors_path = 'config/yolo_anchors.txt'  # anchors

num_classes = 2  # 类别数
anchors = get_anchors(anchors_path)  # anchors列表

input_shape = (416, 416)  # 32的倍数，输入图像

val_split = 0.1  # 训练和验证的比例
np.random.seed(1)
#np.random.shuffle(data)
data=data[1000:10000]
np.random.seed(None)
num_val = int(len(data) * val_split)  # 验证集数量
num_train = len(data) - num_val  # 训练集数量

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',monitor='val_loss', save_weights_only=True,save_best_only=True, period=3)  # 只存储weights，

model = create_model(input_shape, anchors, num_classes,
                         freeze_body=2,
                         weights_path=pretrained_path)
model.compile(optimizer=Adam(lr=1e-3), loss={
    # use custom yolo_loss Lambda layer.
    'yolo_loss': lambda y_true, y_pred: y_pred})  # 损失函数

batch_size = 32  # batch尺寸
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(data[:num_train], batch_size, input_shape, anchors, num_classes),
                    steps_per_epoch=max(1, num_train // batch_size),
                    validation_data=data_generator_wrapper(
                        data[num_train:], batch_size, input_shape, anchors, num_classes),
                    validation_steps=max(1, num_val // batch_size),
                    epochs=20,

                    initial_epoch=0,
                    callbacks=[logging, checkpoint])
model.save(log_dir + 'trained_weights_stage_1.h5')  # 存储最终的参数，再训练过程中，通过回调存储

W0629 22:32:43.141307  9516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0629 22:32:43.142296  9516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0629 22:32:43.152306  9516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0629 22:32:43.153302  9516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0629 22:32:43.164239  9516 

Create YOLOv3 model with 9 anchors and 2 classes.


OSError: Unable to open file (unable to open file: name = 'model/yolo-3.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)